In [ ]:
import glob
import torch
import numpy as np  # use only torch?
import soundfile as sf
# import torch.nn.functional as F
from IPython.display import Audio
torch.set_num_threads(1)
from utils import init_jit_model, STFTExtractor, get_speech_ts, read_audio, state_generator
extractor = STFTExtractor()

## Full audio example

In [ ]:
def collect_speeches(tss, wav):
    speech_chunks = []
    for i in tss:
        speech_chunks.append(wav[i['start']: i['end']])
    return np.concatenate(speech_chunks)

In [ ]:
model = init_jit_model('files/joint_VAD_just_RU_jit_cut_q.pth.tar', 'cpu')  # from yml file

In [ ]:
Audio('files/test_audio_6.wav')

In [ ]:
wav = read_audio('files/test_audio_6.wav')

In [ ]:
torch.__version__

In [ ]:
torch.vstack

In [ ]:
speech_timestamps = get_speech_ts(wav, model, extractor, num_steps=4)  # kill extractor

In [ ]:
sf.write('only_speech.wav', collect_speeches(speech_timestamps, wav), 16000)
Audio('only_speech.wav')

## Stream example

In [ ]:
!ls -laht files/joint_VAD_just_RU_jit_cut_q.pth.tar

In [ ]:
model = init_jit_model('files/joint_VAD_just_RU_jit_cut_q.pth.tar', 'cpu')

In [ ]:
audios_for_stream = glob.glob('files/test*.wav')
len(audios_for_stream)

In [ ]:
for i in state_generator(model, audios_for_stream, extractor, audios_in_stream=2):
    if i:
        print(i)